# Assignment 1 - Propagation Settings

In [1]:
''' 
Copyright (c) 2010-2020, Delft University of Technology
All rigths reserved

This file is part of the Tudat. Redistribution and use in source and 
binary forms, with or without modification, are permitted exclusively
under the terms of the Modified BSD license. You should have received
a copy of the license with this file. If not, please or visit:
http://tudat.tudelft.nl/LICENSE.
'''

import numpy as np
from tudatpy import elements
from tudatpy.io import save2txt
from tudatpy.kernel import constants
from tudatpy.kernel.interface import spice_interface
from tudatpy.kernel.simulation import environment_setup
from tudatpy.kernel.simulation import propagation_setup
from matplotlib import pyplot as plt

In [2]:
# student number: 1244779 --> 1244ABC
A = 7
B = 2
C = 6

simulation_start_epoch = 33.15 * constants.JULIAN_YEAR + A * 7.0 * constants.JULIAN_DAY + \
                            B * constants.JULIAN_DAY + C * constants.JULIAN_DAY / 24.0
simulation_end_epoch = simulation_start_epoch + 344.0 * constants.JULIAN_DAY / 24.0

## Create Environment and Vehicle

In [3]:
###########################################################################
# CREATE ENVIRONMENT ######################################################
###########################################################################

# Load spice kernels.
spice_interface.load_standard_kernels()

# Create body objects.
bodies_to_create = ["Ganymede","Jupiter"]
global_frame_origin = "SSB"
global_frame_orientation = "ECLIPJ2000"
body_settings = environment_setup.get_default_body_settings(
    bodies_to_create, global_frame_origin, global_frame_orientation)    

# Add Ganymede exponential atmosphere 
density_scale_height = 40.0E3
density_at_zero_altitude = 2.0E-9
body_settings.get( "Ganymede" ).atmosphere_settings = environment_setup.atmosphere.exponential( 
        density_scale_height, density_at_zero_altitude)

bodies = environment_setup.create_system_of_bodies(body_settings)



## Propagate Dynamics for various cases

In [5]:
cases = ['case1','case2']

simulation_results_dict = dict()
dependent_variables_dict = dict()
for case in cases:    


    ###########################################################################
    # CREATE ENVIRONMENT ######################################################
    ###########################################################################

    # Load spice kernels.
    spice_interface.load_standard_kernels()

    # Create body objects.
    bodies_to_create = ["Ganymede","Jupiter"]
    if case == 'case1' or case == 'case2':
        global_frame_origin = "Jupiter"
    global_frame_orientation = "ECLIPJ2000"
    body_settings = environment_setup.get_default_body_settings(
        bodies_to_create, global_frame_origin, global_frame_orientation)    
    bodies = environment_setup.create_system_of_bodies(body_settings)
    ###########################################################################
    # CREATE ACCELERATIONS ####################################################
    ###########################################################################

    # Define bodies that are propagated.
    bodies_to_propagate = ["Ganymede"]

    # Define central bodies.
    if case == 'case1' or case == 'case2':
        central_bodies = ["Jupiter"]

    # Define accelerations acting on vehicle.
    if  case == 'case1':
        acceleration_settings_on_body = dict(
            Jupiter=
            [
                 propagation_setup.acceleration.point_mass_gravity()
            ]
        )
    elif  case == 'case2':
        
        acceleration_settings_on_body = dict(
            Jupiter=
            [
                propagation_setup.acceleration.mutual_spherical_harmonic_gravity(4,0,2,2) 
            ]
        )
        

    # Create global accelerations dictionary.
    acceleration_settings = {"Ganymede": acceleration_settings_on_body}
  
    
     # Create acceleration models.
    acceleration_models = propagation_setup.create_acceleration_models(
            bodies, acceleration_settings, bodies_to_propagate, central_bodies)
    ###########################################################################
    # CREATE PROPAGATION SETTINGS #############################################
    ###########################################################################

    # Define initial state.
  

    system_ganymede_state = spice_interface.get_body_cartesian_state_at_epoch(
            target_body_name="Ganymede",
            observer_body_name="Jupiter",
            reference_frame_name="ECLIPJ2000",
            aberration_corrections="NONE",
            ephemeris_time= simulation_start_epoch )
    system_initial_state = system_ganymede_state
    # Save magnitude of perturbations for both cases
    if case == 'case1' or case == 'case2':
        dependent_variables_to_save = [
            propagation_setup.dependent_variable.keplerian_state( "Ganymede", "Jupiter" )
        ]

    # Create propagation settings.
    propagator_settings = propagation_setup.propagator.translational(
        central_bodies,
        acceleration_models,
        bodies_to_propagate,
        system_initial_state,
        simulation_end_epoch,
        output_variables = dependent_variables_to_save
    )

    # Create numerical integrator settings.
    fixed_step_size = 10.0
    integrator_settings = propagation_setup.integrator.runge_kutta_4(
        simulation_start_epoch,
        fixed_step_size
    )

    ###########################################################################
    # PROPAGATE ORBIT #########################################################
    ###########################################################################

    # Create simulation object and propagate dynamics.
    dynamics_simulator = propagation_setup.SingleArcDynamicsSimulator(
        bodies, integrator_settings, propagator_settings)
    
    simulation_results_dict[case] = dynamics_simulator.state_history
    dependent_variables_dict[case] = dynamics_simulator.dependent_variable_history

    ###########################################################################
    # PRINT FINAL PROPAGATION TIME AND STATE ##################################
    ###########################################################################

    final_time_step=list(simulation_results_dict[case].keys())[-1]
    first_time_step=list(simulation_results_dict[case].keys())[0]

    print(
        f"""
    JUICE Propagation Results of {case}.

    Final propagation time of JUICE [s]: {simulation_end_epoch}
    Final Cartesian state of JUICE is [m]: \n{
        simulation_results_dict[case][final_time_step][:]}

        """
    )

    ###########################################################################
    # SAVE RESULTS ############################################################
    ###########################################################################
    
#     save2txt(solution=simulation_result,
#              filename="JUICEPropagationHistory_Q4_" + case + ".dat",
#              directory="./",  # default = "./" 
#              column_names=None,  # default = None 
#              )
    


    JUICE Propagation Results of case1.

    Final propagation time of JUICE [s]: 1051800840.0
    Final Cartesian state of JUICE is [m]: 
[ 7.23453821e+08  7.85342647e+08  4.12504767e+07 -8.00859897e+03
  7.39173212e+03  1.52828162e+02]

        


TypeError: Unable to convert function return value to a Python type! The signature was
	(maximum_degree_body_exerting: int, maximum_order_body_exerting: int, maximum_degree_body_undergoing: int, maximum_order_body_undergoing: int, maximum_degree_central_body: int = 0, maximum_order_central_body: int = 0) -> tudat::simulation_setup::MutualSphericalHarmonicAccelerationSettings

## Pre-process Results

In [ ]:
#PREPROCESSING
#GETTING CONSTANTS
ganymede_gravitational_parameter = body_settings.get( "Ganymede" ).gravity_field_settings.get_gravitational_parameter( )
jupiter_gravitational_parameter = body_settings.get( "Jupiter" ).gravity_field_settings.get_gravitational_parameter( )
ganymede_normalized_c20 = body_settings.get( "Ganymede" ).gravity_field_settings.get_cosine_coefficients( )[2,0]
ganymede_normalized_c22 = body_settings.get( "Ganymede" ).gravity_field_settings.get_cosine_coefficients( )[2,2]
jupiter_normalized_c20 = body_settings.get( "Jupiter" ).gravity_field_settings.get_cosine_coefficients( )[2,0]
jupiter_normalized_c30 = body_settings.get( "Jupiter" ).gravity_field_settings.get_cosine_coefficients( )[3,0]
jupiter_normalized_c40 = body_settings.get( "Jupiter" ).gravity_field_settings.get_cosine_coefficients( )[4,0]
ganymede_reference_radius = body_settings.get( "Ganymede" ).gravity_field_settings.get_reference_radius( )
jupiter_reference_radius = body_settings.get( "Jupiter" ).gravity_field_settings.get_reference_radius
ganymede_normalized_J2 = -ganymede_normalized_c20*np.sqrt(5)
ganymede_normalized_C22 = -ganymede_normalized_c22*np.sqrt(5/12)
jupiter_normalized_J2 = -jupiter_normalized_c20*np.sqrt(5)
jupiter_normalized_J3 = -jupiter_normalized_c30*np.sqrt(7)
jupiter_normalized_J4 = -jupiter_normalized_c40*np.sqrt(9)


a_g_20 = -3*ganymede_gravitational_parameter*ganymede_normalized_J2*ganymede_reference_radius**2
a_g_22 = -3*ganymede_gravitational_parameter*ganymede_normalized_C22*ganymede_reference_radius**2

a_j_20 = -3*jupiter_gravitational_parameter*jupiter_normalized_J2*jupiter_reference_radius**2
a_j_30 = -4*jupiter_gravitational_parameter*jupiter_normalized_J3*jupiter_reference_radius**3
a_j_40 = -5*jupiter_gravitational_parameter*jupiter_normalized_J4*jupiter_reference_radius**4



simulation_result_i = simulation_results_dict[ 'case1' ]

simulation_result_ii = simulation_results_dict[ 'case2' ]




dependent_variables_case1= dependent_variables_dict[ 'case1' ]

dependent_variables_case2= dependent_variables_dict[ 'case2' ]

kepler_elements_case1= np.vstack(list(dependent_variables_case1.values()))[:,0:6]
kepler_elements_case2 = np.vstack(list(dependent_variables_case2.values()))[:,0:6]


semi_major_axis_case1= [ element/1000 for element in kepler_elements_case1[:,0] ]
semi_major_axis_case2 = [ element/1000 for element in kepler_elements_case2[:,0] ]

true_anomaly_case1= [ np.rad2deg( element ) for element in kepler_elements_case1[:,5] ]
true_anomaly_case2 = [ np.rad2deg( element ) for element in kepler_elements_case2[:,5] ]


r_case1= 
r_case2 = 

d_theta = np.subtract(true_anomaly_case2-true_anomaly_i)

## Plot Results

In [ ]:
# Extract time and Kepler elements from dependent variables
kepler_elements = np.vstack(list(dependent_variables.values()))
                            
# Kepler Elements
# 0: semi-major axis
# 1: eccentricity
# 2: inclination
# 3: argument of periapsis
# 4: right ascension of the ascending node
# 5: true anomaly

time = dependent_variables.keys()
time_days = [ t / constants.JULIAN_DAY - simulation_start_epoch / constants.JULIAN_DAY for t in time ]

ganymede_gravitational_parameter = body_settings.get( "Ganymede" ).gravity_field_settings.get_gravitational_parameter( )
ganymede_normalized_c20 = body_settings.get( "Ganymede" ).gravity_field_settings.get_cosine_coefficients( )[2,0]
ganymede_reference_radius = body_settings.get( "Ganymede" ).gravity_field_settings.get_reference_radius( )

ganymede_normalized_J2 = -ganymede_normalized_c20*np.sqrt(5)
a = kepler_elements[1,0]
i = kepler_elements[1,2]
e = kepler_elements[1,1]


time_days = [ t / constants.JULIAN_DAY - simulation_start_epoch / constants.JULIAN_DAY for t in time ]
plt.plot(time_days,  difference_in_car_position_case1)
plt.plot(time_days, difference_in_car_position_case2)
plt.semilogy()
plt.xlabel('Time [day]')
plt.ylabel(r'$\Delta r [m]$')
plt.show()

In [ ]:
###########################################################################
# PLOT RESULTS ############################################################
###########################################################################


                            
# Kepler Elements
# 0: semi-major axis
# 1: eccentricity
# 2: inclination
# 3: argument of periapsis
# 4: right ascension of the ascending node
# 5: true anomaly

time = dependent_variables_i.keys()
time_days = [ t / constants.JULIAN_DAY - simulation_start_epoch / constants.JULIAN_DAY for t in time ]


font_size = 20
                            
plt.rcParams.update({'font.size': font_size}) 

fig, ((ax1, ax2)) = plt.subplots( 2, 1, figsize = (20,17) )

ax1.plot( time_days, semi_major_axis_case1 )
ax1.plot( time_days, semi_major_axis_case2 )
ax1.set_ylabel( 'Semi-major axis [km]' )
ax1.legend(['case 1','case 2'])
ax1.grid()



ax2.scatter( time_days, true_anomaly_case1, s=1 )
ax2.scatter( time_days, true_anomaly_case2, s=1 )
ax2.set_ylabel( 'True Anomaly [deg]' )
ax2.set_yticks(np.arange(0, 361, step=60))
ax2.legend(['case 1','case 2'])
ax2.grid()